In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import random
from sklearn.cross_validation import train_test_split

/usr/local/lib/python3.6/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [0]:
# Install the PyDrive wrapper & import libraries.
# This only needs to be done once per notebook.
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [3]:
# List .txt files in the root.
#
# Search query reference:
# https://developers.google.com/drive/v2/web/search-parameters
listed = drive.ListFile().GetList()
for file in listed:
  print('title {}, id {}'.format(file['title'], file['id']))

title DNN.ipynb, id 1PouCenS2tKofZWjrOo7ke-eQ46p32iyI
title RNN.ipynb, id 19r_dd89G14_x_NqYxACon_LVkg7R1qd0
title sample_phones_with_shuffling.ipynb, id 1DgTzyyf5a7cowx2gNLwylqqoP-l968vh
title sample_phones_with_shuffling.ipynb, id 1_HhSJSMCsTEqpI5BZFAxFyF7k5DxEgTp
title Untitled0.ipynb, id 1GO7GAV5U021FUw79Z6gZ8rS0N4Vzz7Im
title Untitled0.ipynb, id 1c2XTxnRJRhwpfjGVYynbw7rTzl3sT4gs
title Untitled1.ipynb, id 19L2jhVfnalXOvlgFI7TQj4nk0hdc8esZ
title mfcc_alignments.pkl, id 14X-vptKNFo9t-ltnNuXZp_UFHPyaAeA2
title mfcc_alignments.pkl, id 10FsU7gZvkTHyUuhQwUAwsLmEMkIiD4xL
title Untitled folder, id 1N3xA66rADBEFIiNYupzDFYLwKkwleaGV
title sample_phones_with_shuffling.ipynb, id 1tPhS-QoSx-Y9_TwTAUYX3nzzsJqs1vE5
title Colab Notebooks, id 1tebh5kjU9pWe5wmoCGi2oJqtt4VniNuW
title Getting started, id 0B-COj-emT6bUc3RhcnRlcl9maWxl


In [4]:
selection = "mfcc_alignments.pkl"
for file in listed:
  if file['title'] == selection:
    file.GetContentFile(selection)
import os
os.listdir(".")

['datalab',
 '.ipython',
 '.config',
 'mfcc_alignments.pkl',
 '.forever',
 '.rnd',
 '.nv',
 '.cache',
 '.local',
 '.keras']

In [0]:
data=pd.read_pickle("mfcc_alignments.pkl")
labels_list = data["label"]
j=1
for i in range(1,288,6):
    labels_list = labels_list.replace(to_replace=np.arange(i,i+6), value=j)
    j+=1
data["label"] = labels_list

In [0]:
X_input = data["mfcc"]
Y_input = data["label"]

In [0]:
X1 = X_input.tolist()
# Y1 = pd.get_dummies(Y_input.tolist()).values
Y1 = Y_input.tolist()

In [0]:
Y1 = [ele-1 for ele in Y1]

In [0]:
X1 = np.asarray([np.asarray(ele, np.float32) for ele in X1])
Y1 = np.asarray([np.asarray(ele, np.int32) for ele in Y1])

In [0]:
X1 = X1 / X1.max(axis=0)

In [0]:
select_indices = random.sample(range(0,len(X1)), len(X1))
X1 = X1[select_indices]
Y1 = Y1[select_indices]

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X1, Y1, test_size=0.33, random_state=0)

In [0]:
# Parameters
learning_rate = 0.001
training_epochs = 25
batch_size = 1000
display_step = 1

In [0]:
# tf Graph input
X = tf.placeholder("float", [None, 13])
Y = tf.placeholder("float", [None, 48])

In [0]:
feature_columns = [tf.feature_column.numeric_column("x", shape=[13])]

In [19]:
classifier = tf.estimator.DNNClassifier(feature_columns=feature_columns,
                                   hidden_units=[1024, 1024, 1024, 1024, 1024, 1024],
                                   activation_fn=tf.nn.relu,
                                   dropout=0.2,
                                   n_classes=48,
                                   optimizer="Adam")

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmp_m1lkxou', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f44e4a71cc0>, '_task_type': 'worker', '_task_id': 0, '_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [0]:
# Define the training inputs
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": np.array(X_train)},
    y=np.array(y_train),
    num_epochs=10,
    shuffle=True)

In [0]:
test_input_fn = tf.estimator.inputs.numpy_input_fn(
      x={"x": np.array(X_test)},
      y=np.array(y_test),
      num_epochs=1,
      shuffle=False)

In [22]:
classifier.train(input_fn=train_input_fn, steps=20000)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/tmp_m1lkxou/model.ckpt.
INFO:tensorflow:loss = 495.4826, step = 1
INFO:tensorflow:global_step/sec: 87.8107
INFO:tensorflow:loss = 463.00748, step = 101 (1.143 sec)
INFO:tensorflow:global_step/sec: 101.904
INFO:tensorflow:loss = 444.67865, step = 201 (0.979 sec)
INFO:tensorflow:global_step/sec: 102.422
INFO:tensorflow:loss = 721.7291, step = 301 (0.977 sec)
INFO:tensorflow:global_step/sec: 101.815
INFO:tensorflow:loss = 417.92102, step = 401 (0.983 sec)
INFO:tensorflow:global_step/sec: 100.929
INFO:tensorflow:loss = 412.5637, step = 501 (0.993 sec)
INFO:tensorflow:global_step/sec: 100.305
INFO:tensorflow:loss = 457.7221, step = 601 (0.996 sec)
INFO:tensorflow:global_step/sec: 101.252
INFO:tensorflow:loss = 3518.6462, step = 701 (0.988 sec)
INFO:tensorflow:global_step/sec: 100.818
INFO:tensorflow:loss = 435.00183, step = 801 (0.993 sec)
INFO:tensorflow:global_step/sec: 101.11
INFO:tensorflow:lo

INFO:tensorflow:global_step/sec: 101.791
INFO:tensorflow:loss = 456.64825, step = 3101 (0.983 sec)
INFO:tensorflow:global_step/sec: 101.604
INFO:tensorflow:loss = 448.13513, step = 3201 (0.984 sec)
INFO:tensorflow:global_step/sec: 100.703
INFO:tensorflow:loss = 444.23898, step = 3301 (0.991 sec)
INFO:tensorflow:global_step/sec: 100.987
INFO:tensorflow:loss = 435.02216, step = 3401 (0.992 sec)
INFO:tensorflow:global_step/sec: 99.8
INFO:tensorflow:loss = 436.14612, step = 3501 (1.001 sec)
INFO:tensorflow:global_step/sec: 99.915
INFO:tensorflow:loss = 459.54498, step = 3601 (0.999 sec)
INFO:tensorflow:global_step/sec: 101.861
INFO:tensorflow:loss = 451.78955, step = 3701 (0.986 sec)
INFO:tensorflow:global_step/sec: 99.4292
INFO:tensorflow:loss = 457.76392, step = 3801 (1.004 sec)
INFO:tensorflow:global_step/sec: 102.593
INFO:tensorflow:loss = 455.848, step = 3901 (0.972 sec)
INFO:tensorflow:global_step/sec: 99.3785
INFO:tensorflow:loss = 2068.7192, step = 4001 (1.009 sec)
INFO:tensorflow:

INFO:tensorflow:loss = 448.63474, step = 6201 (0.979 sec)
INFO:tensorflow:global_step/sec: 102.192
INFO:tensorflow:loss = 447.17084, step = 6301 (0.977 sec)
INFO:tensorflow:global_step/sec: 102.4
INFO:tensorflow:loss = 452.17496, step = 6401 (0.973 sec)
INFO:tensorflow:global_step/sec: 101.058
INFO:tensorflow:loss = 465.3558, step = 6501 (0.995 sec)
INFO:tensorflow:global_step/sec: 100.062
INFO:tensorflow:loss = 454.9842, step = 6601 (1.000 sec)
INFO:tensorflow:global_step/sec: 101.6
INFO:tensorflow:loss = 462.6089, step = 6701 (0.983 sec)
INFO:tensorflow:global_step/sec: 101.843
INFO:tensorflow:loss = 435.62115, step = 6801 (0.981 sec)
INFO:tensorflow:global_step/sec: 100.801
INFO:tensorflow:loss = 457.5869, step = 6901 (0.991 sec)
INFO:tensorflow:global_step/sec: 101.134
INFO:tensorflow:loss = 441.42487, step = 7001 (0.989 sec)
INFO:tensorflow:global_step/sec: 99.9276
INFO:tensorflow:loss = 448.0948, step = 7101 (1.001 sec)
INFO:tensorflow:global_step/sec: 103.175
INFO:tensorflow:los

INFO:tensorflow:global_step/sec: 101.51
INFO:tensorflow:loss = 449.4563, step = 9401 (0.988 sec)
INFO:tensorflow:global_step/sec: 101.191
INFO:tensorflow:loss = 486.21802, step = 9501 (0.989 sec)
INFO:tensorflow:global_step/sec: 101.579
INFO:tensorflow:loss = 456.77325, step = 9601 (0.987 sec)
INFO:tensorflow:global_step/sec: 99.5632
INFO:tensorflow:loss = 457.97723, step = 9701 (1.002 sec)
INFO:tensorflow:global_step/sec: 98.9393
INFO:tensorflow:loss = 445.48895, step = 9801 (1.007 sec)
INFO:tensorflow:global_step/sec: 98.574
INFO:tensorflow:loss = 449.77466, step = 9901 (1.015 sec)
INFO:tensorflow:global_step/sec: 100.947
INFO:tensorflow:loss = 458.60608, step = 10001 (0.992 sec)
INFO:tensorflow:global_step/sec: 101.321
INFO:tensorflow:loss = 451.67697, step = 10101 (0.988 sec)
INFO:tensorflow:global_step/sec: 101.556
INFO:tensorflow:loss = 460.4077, step = 10201 (0.982 sec)
INFO:tensorflow:global_step/sec: 100.533
INFO:tensorflow:loss = 459.25574, step = 10301 (0.998 sec)
INFO:tenso

INFO:tensorflow:loss = 456.5578, step = 12501 (0.985 sec)
INFO:tensorflow:global_step/sec: 101.885
INFO:tensorflow:loss = 455.58948, step = 12601 (0.982 sec)
INFO:tensorflow:global_step/sec: 101.843
INFO:tensorflow:loss = 436.35464, step = 12701 (0.985 sec)
INFO:tensorflow:global_step/sec: 101.4
INFO:tensorflow:loss = 454.87988, step = 12801 (0.987 sec)
INFO:tensorflow:global_step/sec: 99.078
INFO:tensorflow:loss = 445.70148, step = 12901 (1.009 sec)
INFO:tensorflow:global_step/sec: 101.265
INFO:tensorflow:loss = 457.07825, step = 13001 (0.988 sec)
INFO:tensorflow:global_step/sec: 101.384
INFO:tensorflow:loss = 451.10703, step = 13101 (0.983 sec)
INFO:tensorflow:global_step/sec: 101.586
INFO:tensorflow:loss = 458.66473, step = 13201 (0.987 sec)
INFO:tensorflow:global_step/sec: 101.725
INFO:tensorflow:loss = 457.00604, step = 13301 (0.982 sec)
INFO:tensorflow:global_step/sec: 102.396
INFO:tensorflow:loss = 455.55426, step = 13401 (0.979 sec)
INFO:tensorflow:global_step/sec: 101.658
INFO

INFO:tensorflow:loss = 442.86334, step = 15601 (0.985 sec)
INFO:tensorflow:global_step/sec: 100.495
INFO:tensorflow:loss = 436.68628, step = 15701 (0.994 sec)
INFO:tensorflow:global_step/sec: 102.151
INFO:tensorflow:loss = 454.651, step = 15801 (0.981 sec)
INFO:tensorflow:global_step/sec: 101.946
INFO:tensorflow:loss = 448.22043, step = 15901 (0.983 sec)
INFO:tensorflow:global_step/sec: 101.609
INFO:tensorflow:loss = 470.06702, step = 16001 (0.983 sec)
INFO:tensorflow:global_step/sec: 102.763
INFO:tensorflow:loss = 436.3468, step = 16101 (0.974 sec)
INFO:tensorflow:global_step/sec: 99.1912
INFO:tensorflow:loss = 455.99585, step = 16201 (1.006 sec)
INFO:tensorflow:global_step/sec: 102.091
INFO:tensorflow:loss = 453.8816, step = 16301 (0.978 sec)
INFO:tensorflow:global_step/sec: 102.768
INFO:tensorflow:loss = 455.3573, step = 16401 (0.976 sec)
INFO:tensorflow:global_step/sec: 102.056
INFO:tensorflow:loss = 460.9629, step = 16501 (0.977 sec)
INFO:tensorflow:global_step/sec: 102.206
INFO:t

INFO:tensorflow:loss = 454.67484, step = 18701 (0.984 sec)
INFO:tensorflow:global_step/sec: 101.792
INFO:tensorflow:loss = 456.8372, step = 18801 (0.981 sec)
INFO:tensorflow:global_step/sec: 101.286
INFO:tensorflow:loss = 443.42734, step = 18901 (0.987 sec)
INFO:tensorflow:global_step/sec: 102.18
INFO:tensorflow:loss = 454.148, step = 19001 (0.979 sec)
INFO:tensorflow:global_step/sec: 102.619
INFO:tensorflow:loss = 458.63397, step = 19101 (0.980 sec)
INFO:tensorflow:global_step/sec: 101.62
INFO:tensorflow:loss = 450.8733, step = 19201 (0.981 sec)
INFO:tensorflow:global_step/sec: 101.982
INFO:tensorflow:loss = 450.22986, step = 19301 (0.980 sec)
INFO:tensorflow:global_step/sec: 100.339
INFO:tensorflow:loss = 439.66895, step = 19401 (0.997 sec)
INFO:tensorflow:global_step/sec: 102.704
INFO:tensorflow:loss = 443.2223, step = 19501 (0.973 sec)
INFO:tensorflow:global_step/sec: 102.464
INFO:tensorflow:loss = 459.6831, step = 19601 (0.975 sec)
INFO:tensorflow:global_step/sec: 101.443
INFO:ten

In [23]:
accuracy_score = classifier.evaluate(input_fn=test_input_fn)["accuracy"]

INFO:tensorflow:Starting evaluation at 2018-01-23-19:04:30
INFO:tensorflow:Restoring parameters from /tmp/tmp_m1lkxou/model.ckpt-20000
INFO:tensorflow:Finished evaluation at 2018-01-23-19:04:43
INFO:tensorflow:Saving dict for global step 20000: accuracy = 0.12997317, average_loss = 3.5390096, global_step = 20000, loss = 452.98346


In [24]:
print("\nTest Accuracy: {0:f}\n".format(accuracy_score))


Test Accuracy: 0.129973

